# Environment Demo

In [150]:
import sys
import os
import importlib
import numpy as np
import time

# Add the parent directory to the Python path
sys.path.append(os.path.abspath('../'))
import environment 
importlib.reload(environment)
import utils
importlib.reload(utils)

<module 'utils' from '/Users/lorenzoleuzzi/Documents/GitHub/lifelong_evolutionary_swarms/utils.py'>

## Robot features

- 25 cm diameter
- compass
- 360 vision sensor and object reconition in range 100cm
- comunication between others robots
- ability to pick up stuff (in they're in the same position of the object)
- holonomic motion (every directions)
- maximum velocity: 200 cm/s
- maximum acceleration: 400 cm/s²
- maximum distance covered in a single move: 1/2 acceleration * tstep, 8cm with tstep = 200 ms 

Simulation measurements: everything is divide by the agent size which is reperesented as point in the arena

In [130]:
f"{environment.SIMULATION_ROBOT_SIZE}, {environment.SIMULATION_SENSOR_RANGE}, {environment.TIME_STEP}, {environment.SIMULATION_MAX_WHEEL_VELOCITY}"

'1.0, 4.0, 0.1, 2.0'

We are in a continuous 2D environment (no physics), a robot possesses the capability to navigate in any direction, covering any distance up to a defined maximum per step. Additionally, the robot can pick up and deposit an object objects when is underneath or when it reached an edge, resepctively.

The robots are equipped with sensory equipment capable of identifying nearby entities. A "neighbor" is characterized by a tuple comprising the type of object, the distance to it, and its relative direction. Accordingly, each robot maintains a list of such tuples for a predefined fixed number of neighboring entities.

## Arena

5m x 5m with robots and colored objects 

In [131]:
environment.SIMULATION_ARENA_SIZE 

20.0

## Objective
List of (color_id, edge). The robots must pick up the objects and deposit them in right position. The deposit area is in an edge of the arena.

In [132]:
# Example
[(environment.RED, environment.UP), (environment.GREEN, environment.DOWN)]

[(3, 0), (5, 180)]

In [40]:
environment.REWARD_PICK, environment.REWARD_DROP

(1, 2)

## Environment initialization

In [133]:
initial_setting = {
   'agents': np.array([[0, 6], [0, 7], [0, 8]], dtype=float),
   'headings': np.array([environment.DOWN, environment.DOWN, environment.DOWN], dtype=float),
   'blocks': np.array([[10, 16], [12, 7], [10, 5], [10, 9], [9, 7]], dtype=float),
   'colors': np.array([environment.RED, environment.RED, environment.BLUE, environment.GREEN, environment.RED], dtype=int)
}

env = environment.SwarmForagingEnv(objective = [(environment.RED, environment.UP)],
                   size = environment.SIMULATION_ARENA_SIZE, 
                   n_agents = 3, 
                   n_blocks = 5,
                   n_neighbors = 4,
                   sensor_range = environment.SIMULATION_SENSOR_RANGE,
                   sensor_angle = 360,
                   max_wheel_velocity = environment.SIMULATION_MAX_WHEEL_VELOCITY,
                   sensitivity = 0.5,
                   initial_setting = initial_setting)
initial_state, _ = env.reset() # Initial state
env.render()
initial_state

. . . . . . 0 1 2 . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . O . . . . . . . . . . . .
. . . . . O . . . O . . . . . . O . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . O . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .


[{'neighbors': array([[  1.,   0.,   0.],
         [  2.,   1., 270.],
         [  2.,   2., 270.],
         [  0.,   0.,   0.]]),
  'heading': 180.0,
  'carrying': -1},
 {'neighbors': array([[  1.,   0.,   0.],
         [  2.,   1.,  90.],
         [  2.,   1., 270.],
         [  0.,   0.,   0.]]),
  'heading': 180.0,
  'carrying': -1},
 {'neighbors': array([[ 1.,  0.,  0.],
         [ 2.,  1., 90.],
         [ 2.,  2., 90.],
         [ 0.,  0.,  0.]]),
  'heading': 180.0,
  'carrying': -1}]

## Testing

In [55]:
cumulative_reward = 0

In [72]:
action = env.action_space.sample()
action = (environment.MOVE_UP, environment.MOVE_UP * np.array([5]), environment.ROTATE_POSITIVE)
# action = ([2.2355506, -3.99576594, 3.99998646], [2.2355506, -3.99576594, 3.99998646], [2.2355506, -3.99576594, 3.99998646])
next_state, reward, done, _, _ = env.step(action)
env.render()
cumulative_reward += reward
reward, cumulative_reward

. . . . . . 0 . 2 . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . 1 . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . O . . . O . . . . . . O . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . O . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .


(0.0556, 1.0556)

## Random solver

In [217]:
steps = 0
total_reward = 0
while True:
    action = env.action_space.sample()
    next_state, reward, done, _, info = env.step(action)
    steps += 1
    total_reward += reward
    if done:
        break

f"Done in {steps} steps with reward {total_reward}, info: {info}"

"Done in 184865 steps with reward -9718.344300000048, info: {'completed': [(4, 3, 0), (0, 3, 2), (1, 3, 0)]}"

## Profiling

about 2000 steps per second

In [229]:
def profile():
    total_time = 0
    for i in range(2000):
        action = env.action_space.sample()
        start = time.time()
        next_state, reward, done, _, _ = env.step(action)
        end = time.time()
        total_time += end - start
    return total_time
profile()

0.9050180912017822

In [220]:
import cProfile
cProfile.run('profile()', sort='cumtime')

         886071 function calls (876071 primitive calls) in 1.860 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    1.860    1.860 {built-in method builtins.exec}
        1    0.000    0.000    1.860    1.860 <string>:1(<module>)
        1    0.017    0.017    1.860    1.860 1416959497.py:1(profile)
     2000    0.246    0.000    1.342    0.001 environment.py:371(step)
86824/76824    0.126    0.000    0.716    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
     2000    0.008    0.000    0.497    0.000 tuple.py:87(sample)
     8000    0.012    0.000    0.488    0.000 tuple.py:112(<genexpr>)
     6000    0.148    0.000    0.477    0.000 box.py:184(sample)
    38868    0.035    0.000    0.396    0.000 <__array_function__ internals>:177(norm)
    38868    0.173    0.000    0.323    0.000 linalg.py:2357(norm)
    28868    0.057    0.000    0.285    0.000 environment.py